In [ ]:
from glob import glob
import platform
import os
if platform.system()=='Linux':
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/My Drive/')
  sep='/'
else:
  sep='\\'
print(os.getcwd())

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
import sqlite3 as lite

In [ ]:
con = lite.connect('Team3/dbmsproj.sqlite')

In [ ]:
def transact(con, sql):
    try:
        con.execute(sql)
        con.commit()
    except Exception as e:
        con.rollback()
        print(e)

In [ ]:
import pandas as pd

**READING DATA FROM THE SPREADSHEET**

In [ ]:
bb = pd.read_excel("Team3/transaction_data.xlsx",sheet_name='bankbalance')
Userz = pd.read_excel("Team3/transaction_data.xlsx",sheet_name='Users')
catgor = pd.read_excel("Team3/transaction_data.xlsx",sheet_name='Category')
budgett = pd.read_excel("Team3/transaction_data.xlsx",sheet_name='Budget')


**CREATING TABLE Users**

In [ ]:
con.execute("DROP TABLE IF EXISTS Users")
con.execute("""CREATE TABLE IF NOT EXISTS Users (
                username TEXT PRIMARY KEY,
                name TEXT NOT NULL,
                email TEXT UNIQUE CHECK (email LIKE '_%@_%._%'),
                pin TEXT
            )""")
con.commit()


In [ ]:
con.execute("Drop table if exists Category")
con.execute("""CREATE TABLE if not exists Category (
    category_id text primary key,
    category_name TEXT NOT NULL
);
""")
con.commit()

In [ ]:
con.execute("Drop table if exists Budget")
con.execute("""CREATE TABLE if not exists Budget (
    budget_id INTEGER PRIMARY KEY autoincrement,
    username text,
    category_id text,
    budget_amount REAL
    ,total_amount real,
    start_date text,
    end_date text,
    FOREIGN KEY (username) REFERENCES Users(username),
    FOREIGN KEY (category_id) REFERENCES Categories(category_id)
);

""")
con.commit()

In [ ]:
con.execute("drop table if exists balance")
con.execute("""CREATE TABLE if not exists balance (
    bankbalance_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username text,
    bankbalance int check (bankbalance>=0),
    FOREIGN KEY (username) REFERENCES Users(username)
);
""")
con.commit()

In [ ]:
import hashlib
def encrypt(pin):
  return hashlib.sha256(str(pin).encode()).hexdigest()

In [ ]:
for row in Userz.itertuples(index=False):
    encrypted_pin = encrypt(row.pin)
    con.execute("INSERT INTO Users (username, name, email, pin) VALUES (?, ?, ?, ?)",
                   (row.username, row.name, row.email, encrypted_pin))

IntegrityError: UNIQUE constraint failed: Users.username

In [ ]:
con.commit()

In [ ]:
con.execute("select * from Users").fetchall()

# what is going on here
"""for row in userz.itertuples(index=False): This line starts a loop that iterates over each row in the userz DataFrame.
 itertuples() is a Pandas function that returns an iterator yielding namedtuples for each row in the DataFrame.
  Setting index=False ensures that the index of the DataFrame is not included in the tuples.
con.execute("INSERT INTO Users VALUES (?, ?, ?)", row): Within each iteration of the loop, this line executes an SQL INSERT statement
. It inserts a row into the "Users" table using the values from the current row of the DataFrame (row).
 The question marks (?) are placeholders for the values to be inserted.
 This is a parameterized query where values from the DataFrame row are substituted into the placeholders."""


In [ ]:
for row in catgor.itertuples(index=False):
    con.execute("INSERT INTO category VALUES (?,?)", row)
con.commit()

In [ ]:
con.execute("select * from category").fetchall()

In [ ]:
for row in bb.itertuples(index=False):
    con.execute("INSERT INTO balance(username,bankbalance) VALUES (?,?)", row)
con.execute("select * from balance").fetchall()

In [ ]:
con.commit()

In [ ]:
import pytz
from datetime import datetime

# Get current time in Delhi timezone
delhi_timezone = pytz.timezone('Asia/Kolkata')
delhi_time = datetime.now(delhi_timezone).strftime('%H:%M:%S')

con.execute("DROP TABLE IF EXISTS transactions")
con.execute("""CREATE TABLE IF NOT EXISTS transactions (
    transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
    senders_id TEXT NOT NULL,
    receivers_id TEXT NOT NULL,
    amount INTEGER CHECK(amount > 0),
    date TEXT DEFAULT (DATE(CURRENT_TIMESTAMP)),
    time TEXT DEFAULT '{}',
    category_id TEXT,
    FOREIGN KEY (senders_id) REFERENCES Users(username) ON DELETE RESTRICT,
    FOREIGN KEY (receivers_id) REFERENCES Users(username) ON DELETE RESTRICT,
    FOREIGN KEY (category_id) REFERENCES Category(category_id)
);
""".format(delhi_time))

con.commit()

#current timestamp provides time and date, having date(timestamp) returns only the current date, same for time
#fk connects a column in the given table to another tables pk, this means u can access cols of the other connected table
#association of tables
#on insert restrict allows us to restrict change/insertion/deletion in the child table that violates the foreign key constraint

# a trigger works during crud ops, here the triggers r raised if the username in transactions is not present in users username
# tried using on insert restrict and DEFERRABLE INITIALLY DEFERRED didnt work therefore chose a trigger

#raise(abort) raises an error saying invalid id, of type abort(which halts the current sql statement and rollsback to the)
#previous state,

#SELECT 1 is to check for the existence of a row that meets certain conditions but you don't need to retrieve any actual data from
#the table. It's a way to check whether a condition is true without incurring the overhead of retrieving any specific columns.

In [ ]:
con.execute("""CREATE TRIGGER if not exists update_balance
AFTER INSERT ON transactions
BEGIN
    UPDATE balance
    SET bankbalance = bankbalance - NEW.amount
    WHERE username = NEW.senders_id;

    UPDATE balance
    SET bankbalance = bankbalance + NEW.amount
    WHERE username = NEW.receivers_id;


END;
""")

Use cases of our transaction system.
1. send and receive transactions, the amount is automatically deducted from the sender and increased in the receiver
2. to send and receive transactions, the amount is to be present in the balance of the sender and the pin entered must match the pin stored in the database(prototype)
3. the database also checks if the sender and receiver are both registered in the users table
4. you can get a users bankbalance and transaction history by entering the pin
5. ask for username and password while logging in


In [ ]:
def authenticate_user(username, pin):
    user = con.execute("SELECT username FROM Users WHERE username=? AND (pin=? OR pin IS NULL)", (username,pin)).fetchone()
    if user:
        return user[0]
    else:
        return None


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Email configuration
EMAIL_ADDRESS = 'atheendreramesh@gmail.com'
EMAIL_PASSWORD = 'ddjq gtlz kgtk ggtk'
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587

def send_email(sender_email, receiver_email, subject, body):
    msg = MIMEMultipart('alternative')
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'html'))

    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as s:
        s.starttls()
        s.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        s.send_message(msg)

def transact2(senders_id):
    receivers_id = input("Please enter the receiver's username: ")
    amount = int(input("Please enter the amount: "))

    if senders_id != receivers_id:
        receiver_exists = con.execute("SELECT 1 FROM Users WHERE username=?", (receivers_id,)).fetchone()
        #makes sure ur not sending it to the same user
        if receiver_exists:
            try:
                con.execute("BEGIN")
                con.execute("INSERT INTO transactions (senders_id, receivers_id, amount) VALUES (?, ?, ?)",
                            (senders_id, receivers_id, amount))
                con.commit()
                print("Transaction successful.")

                sender_email = con.execute("SELECT email FROM Users WHERE username=?", (senders_id,)).fetchone()[0]
                receiver_email = con.execute("SELECT email FROM Users WHERE username=?", (receivers_id,)).fetchone()[0]


                sender_subject = "Transaction Successful"
                sender_body = f"Your transaction of {amount} to {receivers_id} is successful."
                send_email(EMAIL_ADDRESS, sender_email, sender_subject, sender_body)


                receiver_subject = "You received a transaction"
                receiver_body = f"You received {amount} from {senders_id}."
                send_email(EMAIL_ADDRESS, receiver_email, receiver_subject, receiver_body)

            except Exception as e:
                con.rollback()
                print(f"error: {e}")
                sender_email = con.execute("SELECT email FROM Users WHERE username=?", (senders_id,)).fetchone()[0]
                sender_subject = "Transaction Failed"
                sender_body = "Your transaction failed. Please try again later."
                send_email(EMAIL_ADDRESS, sender_email, sender_subject, sender_body)
        else:
            print("Receiver's username does not exist.")
    else:
        print("Cannot transact to the same sender and receiver.")


In [ ]:
def login():
  username=input("pls enter username: ")
  pin=encrypt(int(input("pls input pin: ")))
  x=authenticate_user(username,pin)
  if x:
    print("log in successful.")
    return username
  else:
    print("invalid username or pin.")

senders_id=login()

In [ ]:
def transaction(senders_id):
    pin = input("Enter your 6-digit PIN: ")
    pin=encrypt(pin)
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)



In [ ]:
def getbankbalance(senders_id):
    pin = input("Please enter your 6-digit PIN: ")
    pin=encrypt(pin)
    # Authenticate user
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        result = con.execute("SELECT bankbalance FROM balance WHERE username = ?", (senders_id,)).fetchone()
        if result:
            return result[0]
        else:
            return "User balance not found"
    else:
        return "Authentication failed. Incorrect username or PIN."
print("Your bank balance is:", getbankbalance(senders_id))


Please enter your 6-digit PIN: 130505
Your bank balance is: 93697


In [ ]:
def transaction_history():
    pin = input("Please enter your 6-digit PIN: ")
    pin=encrypt(pin)

    user_id = authenticate_user(senders_id,pin)
    if user_id:
        result = con.execute("SELECT senders_id, receivers_id, amount, date, time FROM transactions WHERE senders_id = ? OR receivers_id = ?", (senders_id,senders_id )).fetchall()
        if result:
            for transaction in result:
                print(f"Sender: {transaction[0]}, Receiver: {transaction[1]}, Amount: {transaction[2]}, Date: {transaction[3]}, Time: {transaction[4]}")
        else:
            print(f"No transactions found for user {senders_id}")
    else:
        print("Authentication failed. Incorrect username or PIN.")

transaction_history()


Please enter your 6-digit PIN: 130505
No transactions found for user Atheendre_ram


# ***BUDGET STARTS HERE***

In [ ]:
con.execute("select * from Budget").fetchall()
z= budgett.iloc[:,:4]
z
for row in z.itertuples(index=False):
    con.execute("INSERT INTO Budget(username,category_id,budget_amount,total_amount,start_date,end_date) VALUES (?,?,?,?,'1-04-2024','30-04-20204')", row)
con.execute("select * from Budget").fetchall()

In [ ]:
cursor = con.cursor()

# Define the receiver ids and their corresponding categories
receiver_categories = {
    'AmazedByDeals2023': 'SHP',
    'FlipkartFanatic101': 'SHP',
    'FoodieBangalore99': 'F&B',
    'UberUser_BLR123': 'TRVL',
    'MetroCard_1234567890': 'TRVL',
    'PolarBear_Environmental': 'F&B',
    'Zomatoholic20': 'F&B',
    'Jio_SIM_1234567890': 'PRE',
    'Netflix_Buff': 'CE',
    'TravelExplorer55': 'TRVL',
    'LPG_Consumer_12345': 'DOM',
    'MingosFan_BLR': 'F&B',
    'ApolloHealth_12345': 'MED',
    'Bigb_2342': 'FOOD'
    }

# Define a function to check if budget exceeded
def check_budget(sender_id, category_id, amount):
    # Fetch budget amount for the sender and category
    cursor.execute("SELECT budget_amount FROM Budget WHERE username = ? AND category_id = ?", (sender_id, category_id))
    budget_row = cursor.fetchone()
    if budget_row and amount > budget_row[0]:
        return True  # Budget exceeded
    else:
        return False  # Budget not exceeded

# Define a function to handle the trigger logic
def check_budget_trigger(sender_id, receiver_id, amount):
    # Check if receiver_id is in the predefined list
    if receiver_id in receiver_categories:
        rec_category = receiver_categories[receiver_id]
        # Check if the sender has set a budget for this category and if the transaction amount exceeds it
        if check_budget(sender_id, rec_category, amount):
            # Handle budget exceeded scenario (you can log or take any other action here)
            # For simplicity, I'm just printing a message
            print("Budget Exceeded!")

# Example usage
sender_id = 'Akshaya_akshu'
receiver_id = 'MetroCard_1234567890'
amount = 5000
print(check_budget_trigger(sender_id, receiver_id, amount))

In [ ]:
#Apeksha_appu', 'Harish878', 121, '2024-04-26', '05:47:00', 'CE'
# Create a cursor object
cursor = con.cursor()

# Define the receiver ids and their corresponding categories
receiver_categories = {
    'AmazedByDeals2023': 'SHP',
    'FlipkartFanatic101': 'SHP',
    'FoodieBangalore99': 'F&B',
    'UberUser_BLR123': 'TRVL',
    'MetroCard_1234567890': 'TRVL',
    'PolarBear_Environmental': 'F&B',
    'Zomatoholic20': 'F&B',
    'Jio_SIM_1234567890': 'PRE',
    'Netflix_Buff': 'CE',
    'TravelExplorer55': 'TRVL',
    'LPG_Consumer_12345': 'DOM',
    'MingosFan_BLR': 'F&B',
    'ApolloHealth_12345': 'MED'
}

# Define a function to check if budget exceeded
def check_budget(sender_id, category_id, amount):
    # Fetch budget amount for the sender and category
    cursor.execute("SELECT budget_amount FROM Budget WHERE username = ? AND category_id = ?", (sender_id, category_id))
    budget_amount = cursor.fetchone()
    if budget_amount and amount > budget_amount[0]:
        print(budget_amount)
        print(amount)
        return True  # Budget exceeded
    else:
        return False  # Budget not exceeded

# Define a function to handle the trigger logic
def check_budget_trigger(sender_id, receiver_id, amount, category_id):
    # Check if receiver_id is in the predefined list
    if receiver_id in receiver_categories:
        rec_category = receiver_categories[receiver_id]
        # Check if the sender has set a budget for this category and if the transaction amount exceeds it
        if check_budget(sender_id, rec_category, amount):
            # Handle budget exceeded scenario (you can log or take any other action here)
            # For simplicity, I'm just printing a message
            print("Budget Exceeded!")
            return False  # Budget exceeded, don't insert into transactions table
    return True  # Budget not exceeded, proceed with insertion

# Example usage
sender_id = 'Subbu'
receiver_id = 'Jio_SIM_1234567890'
amount = 30
category_id = 'PRE'

if check_budget_trigger(sender_id, receiver_id, amount, category_id):
    # Insert into transactions table if conditions are true
    con.execute("INSERT INTO transactions (senders_id, receivers_id, amount, category_id) VALUES (?, ?, ?, ?)",
                (sender_id, receiver_id, amount, category_id))
    x =con.execute(f"select budget_amount from Budget where username  = '{sender_id}' and  category_id = '{category_id}'").fetchone()[0]
    x-=amount
    con.execute(f"update Budget set budget_amount = {x} where username = '{sender_id}' and category_id = '{category_id}'")
    y  =con.execute(f"select total_amount from Budget where username ='{sender_id}' and category_id = '{category_id}'").fetchone()[0]
    y+=amount
    con.execute(f"update Budget set total_amount = {y} where username ='{sender_id}' and category_id = '{category_id}'")

    con.commit()
    print("Transaction inserted successfully!")
else:
    print("Transaction not inserted due to budget exceeded.")
con.commit()

In [ ]:
con.execute("select * from  transactions").fetchall()

In [ ]:
con.execute("select * from Balance").fetchall()

In [ ]:
con.close()

In [ ]:
def check_budget(sender_id, category_id, amount):
    # Fetch budget amount for the sender and category
    budget_amount=con.execute(f"SELECT budget_amount FROM Budget WHERE username ={sender_id} AND category_id ={category_id}").fetchone()

    print(budget_amount)
    print(amount)
    if budget_amount and amount > budget_amount[0]:
        print(budget_amount)
        print(amount)
        return True  # Budget exceeded
    else:
        return False  # Budget not exceeded
check_budget('Akshaya_akshu','TRVL','3000')

In [ ]:
con.execute("SELECT budget_amount FROM Budget WHERE username ='Akshaya_akshu' AND category_id ='TRVL'").fetchall()

In [ ]:
con.execute("select budget_amount from Budget where username = 'Apeksha_appu'").fetchall()

In [ ]:
con.execute("select * from Budget").fetchall()

In [ ]:
sender_id = 'shubhangi_shiv'
y =con.execute(f"select * from Budget where username=  '{sender_id}'").fetchall()

In [ ]:
y

In [ ]:
sender_id ='kaveri99'
category_id='MED'
x =con.execute(f"select budget_amount from Budget where username ='{sender_id}' and category_id =' {category_id}'")
# x-=amount
# con.execute(f"update Budget set budget_amount = x where sender_id = '{sender_id} and category_id = '{category_id}'")
# y  =con.execute(f"select total_amount from Budget where username ={sender_id} and category_id = {category_id}").fetchone()[0]
# y+=amount
# con.execute(f"update Budget set total_amount = {y} where username ={sender_id} and category_id = {category_id}")


In [ ]:
x.fetchone()

In [ ]:
con.execute("select * from Budget").fetchall()

In [ ]:
x

In [ ]:
sql = f"select budget_amount from Budget where username ='{sender_id}' and category_id =' {category_id}'"
con.execute(sql).fetchall()

In [ ]:
x.fetchall()